In [1]:
import sys
sys.path.insert(0, './..')
sys.path.insert(0, '../data')

import matplotlib.pyplot as plt
import numpy as np
import plots as pl
from models import model
import torch
from utils import dev, load_data, classification
from torchvision import datasets, transforms
from models import eval
import dill
from robustness1 import model_utils, datasets, train, defaults
from robustness1.datasets import CIFAR

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

/mnt/qb/home/bethge/dschultheiss/AdversarialDecomposition/robustness1/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


# Load CIFAR model

In [2]:
ds = CIFAR('../data/cifar-10-batches-py')
classifier_model = ds.get_model('resnet50', False)
model_natural = model.cifar_pretrained(classifier_model, ds)

resume_path = './models/cifar_models/nat_diff.pt'
checkpoint = torch.load(resume_path, pickle_module=dill, map_location=torch.device(dev()))

state_dict_path = 'model'
if not ('model' in checkpoint):
    state_dict_path = 'state_dict'
sd = checkpoint[state_dict_path]
sd = {k[len('module.'):]:v for k,v in sd.items()}
model_natural.load_state_dict(sd)
model_natural.to(dev())
model_natural.double()
model_natural.eval()

classifier_model = ds.get_model('resnet50', False)
model_robust = model.cifar_pretrained(classifier_model, ds)

resume_path = './models/cifar_models/rob_diff.pt'
checkpoint = torch.load(resume_path, pickle_module=dill, map_location=torch.device(dev()))

state_dict_path = 'model'
if not ('model' in checkpoint):
    state_dict_path = 'state_dict'
sd = checkpoint[state_dict_path]
sd = {k[len('module.'):]:v for k,v in sd.items()}
model_robust.load_state_dict(sd)
model_robust.to(dev())
model_robust.double()
model_robust.eval()

cifar_pretrained(
  (normalizer): InputNormalize()
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): SequentialWithArgs(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05,

# Evaluate model

In [14]:
ds = CIFAR('../data/')
m_nat, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path='./models/cifar_models/nat_diff.pt')
m_rob, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path='./models/cifar_models/rob_diff.pt')
train_loader, val_loader = ds.make_loaders(batch_size=128, workers=8)

# # Create a cox store for logging
# out_store = cox.store.Store('./nat')

# # Hard-coded base parameters
# train_kwargs = {
#     'out_dir': "./rob/train_out",
#     'eval_only': 1,
#     'adv_eval': 1,
#     'constraint': '2',
#     'eps': 0.5,
#     'attack_lr': 1.5,
#     'attack_steps': 20
# }
# train_args = Parameters(train_kwargs)

# # Fill whatever parameters are missing from the defaults
# train_args = defaults.check_and_fill_args(train_args, defaults.CONFIG_ARGS, CIFAR)
# train_args = defaults.check_and_fill_args(train_args, defaults.PGD_ARGS, CIFAR)


# # Train a model
# train.eval_model(train_args, m_rob, val_loader, store=out_store)


# Create a cox store for logging
out_store = cox.store.Store('./rob')

# Hard-coded base parameters
train_kwargs = {
    'out_dir': "./rob/train_out",
    'eval_only': 1,
    'adv_eval': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 1.5,
    'attack_steps': 20
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args, defaults.CONFIG_ARGS, CIFAR)
train_args = defaults.check_and_fill_args(train_args, defaults.PGD_ARGS, CIFAR)

# Train a model
train.eval_model(train_args, m_rob, val_loader, store=out_store)




Custom model
=> loading checkpoint './models/cifar_models/nat_diff.pt'
=> loaded checkpoint './models/cifar_models/nat_diff.pt' (epoch 111)
Custom model
=> loading checkpoint './models/cifar_models/rob_diff.pt'
=> loaded checkpoint './models/cifar_models/rob_diff.pt' (epoch 150)
==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified
Logging in: /mnt/qb/home/bethge/dschultheiss/AdversarialDecomposition/rob/05701b23-7dfe-478a-b339-997a93556188


Val Epoch:0 | Loss 0.4375 | NatPrec1 85.430 | NatPrec5 99.410 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:03<00:00, 25.98it/s]
Val Epoch:0 | Loss 0.9288 | AdvPrec1 65.070 | AdvPrec5 98.180 | Reg term: 0.0 ||: 100%|██████████| 79/79 [02:18<00:00,  1.75s/it]


{'epoch': 0,
 'nat_prec1': tensor(85.4300, device='cuda:0'),
 'adv_prec1': tensor(65.0700, device='cuda:0'),
 'nat_loss': 0.4375362557411194,
 'adv_loss': 0.9287609119415283,
 'train_prec1': nan,
 'train_loss': nan,
 'time': 142.0380403995514}

# Make evenly distributed dataset with true samples only

In [ ]:
images, labels = load_data(10000, random=False, train=False)
correct_classes = np.full(labels.shape, True)

for model_id in range(5):
    md = model.madry_diff()
    md.load_state_dict(torch.load('./models/natural_'+str(model_id)+'.pt', map_location=torch.device(dev())))
    md.to(dev())
    md.eval()
    classes = md(images).argmax(1)
    correct_classes[~(classes==labels).cpu()]=False

for model_id in range(5):
    md = model.madry_diff()
    md.load_state_dict(torch.load('./models/robust_'+str(model_id)+'.pt', map_location=torch.device(dev())))
    md.to(dev())
    md.eval()
    classes = md(images).argmax(1)
    correct_classes[~(classes==labels).cpu()]=False

images = images[correct_classes]
labels = labels[correct_classes]

data = {
    'images': images.cpu(),
    'labels': labels.cpu()
}

save_path = '/home/bethge/dschultheiss/AdversarialDecomposition/data/MNIST/stable_data.npy'
# np.save(save_path, data)

# Concatenate data

In [7]:
model_id = 0
n_batches = 25
model_type = "natural"

data = np.load('./data/' + model_type + str(model_id) + '_0.npy', allow_pickle=True).item()
advs = data['advs']
classes = data['adv_class']
pert_lengths = data['pert_lengths']
dirs = data['dirs']
images = data['images']
labels = data['labels']


for batch_n in np.arange(1,n_batches):
    path = './data/'+ model_type + str(model_id) + '_' + str(batch_n) + '.npy'
    data = np.load(path, allow_pickle=True).item()
    advs = np.concatenate((advs, data['advs']), axis=0)
    classes = np.concatenate((classes, data['adv_class']), axis=0)
    pert_lengths = np.concatenate((pert_lengths, data['pert_lengths']), axis=0)
    dirs = np.concatenate((dirs, data['dirs']), axis=0)
    images = np.concatenate((images, data['images']), axis=0)
    labels = np.concatenate((labels, data['labels']), axis=0)

In [8]:
pert_lengths[pert_lengths==0] = np.nan
sort_idx = np.argsort(pert_lengths,axis=-1)
for i, idx in enumerate(sort_idx):
    pert_lengths[i] = pert_lengths[i,idx]
    dirs[i] = dirs[i,idx]
    advs[i] = advs[i,idx]
    classes[i] = classes[i,idx]

In [9]:
data = {
    'advs': advs,
    'dirs': dirs,
    'adv_class': classes,
    'pert_lengths': pert_lengths,
    'images': images,
    'labels': labels,
}
save_path = '/home/bethge/dschultheiss/AdversarialDecomposition/data/' + model_type + '_' + str(model_id)+ '.npy'
np.save(save_path, data)